In [1]:
import pandas as pd
import contractions
import fasttext
from nltk.tokenize import word_tokenize
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/mahdi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/mahdi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mahdi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/mahdi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
messages_df = pd.read_pickle("messages_scraped.pkl")

In [3]:
messages_df = messages_df[(messages_df['message'].isnull() == False) & (messages_df['user_name'].isnull() == False)]
messages_df = messages_df[(messages_df['message'].values == '') == False]
messages_df.index = range(len(messages_df))

In [4]:
for col in messages_df.columns:
    print(col, (messages_df[col].values == '').sum())

message 0
user_name 0


In [5]:
messages_df['no_contract'] = messages_df['message'].apply(lambda x: [contractions.fix(word) for word in x.split()])

In [6]:
messages_df['no_contract_str'] = [' '.join(map(str, l)) for l in messages_df['no_contract']]

In [7]:
# less accurate model
pretrained_model = "lid.176.ftz"

model = fasttext.load_model(pretrained_model)
langs = []
for sent in messages_df['no_contract_str']:
    lang = model.predict(sent)[0]
    langs.append(str(lang)[11:13])
messages_df['langs'] = langs

In [8]:
messages_df[messages_df['langs'] == 'en']

,message,user_name,no_contract,no_contract_str,langs
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en
...,...,...,...,...,...
1958,\n5 years of 14 to 17 is unsustainable (unles...,Only,"[5, years, of, 14, to, 17, is, unsustainable, ...",5 years of 14 to 17 is unsustainable (unless c...,en
1959,"I don't think the talk we had recently , of th...",Cowmangav,"[I, do not, think, the, talk, we, had, recentl...","I do not think the talk we had recently , of t...",en
1960,We are slow at responding though too many thou...,Only,"[We, are, slow, at, responding, though, too, m...",We are slow at responding though too many thou...,en
1961,Oh great.Now Brussels will allow state aid fo...,Cowmangav,"[Oh, great.Now, Brussels, will, allow, state, ...",Oh great.Now Brussels will allow state aid for...,en


In [9]:
messages_df['tokenized'] = messages_df['no_contract_str'].apply(word_tokenize)
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]"


In [10]:
messages_df['lower'] = messages_df['tokenized'].apply(lambda x: [word.lower() for word in x])
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]"


In [11]:
punc = string.punctuation
messages_df['no_punc'] = messages_df['lower'].apply(lambda x: [word for word in x if word not in punc])
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower,no_punc
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,...","[why, no, milk, price, tracker, thread, we, ha..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno...","[christ, no, its, bad, enough, knowing, what, ..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, be, interesting, t..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]","[looks, like, you, just, started, it]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]","[grr, lower]"


In [12]:
stop_words = set(stopwords.words('english'))
messages_df['stopwords_removed'] = messages_df['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower,no_punc,stopwords_removed
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,...","[why, no, milk, price, tracker, thread, we, ha...","[milk, price, tracker, thread, beef, lamb, fee..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno...","[christ, no, its, bad, enough, knowing, what, ...","[christ, bad, enough, knowing, getting, would,..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, be, interesting, t...","[would, willing, interesting, see, variations,..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]","[looks, like, you, just, started, it]","[looks, like, started]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]","[grr, lower]","[grr, lower]"


In [13]:
messages_df['pos_tags'] = messages_df['stopwords_removed'].apply(nltk.tag.pos_tag)
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,...","[why, no, milk, price, tracker, thread, we, ha...","[milk, price, tracker, thread, beef, lamb, fee...","[(milk, NN), (price, NN), (tracker, NN), (thre..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno...","[christ, no, its, bad, enough, knowing, what, ...","[christ, bad, enough, knowing, getting, would,...","[(christ, NN), (bad, JJ), (enough, RB), (knowi..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, be, interesting, t...","[would, willing, interesting, see, variations,...","[(would, MD), (willing, JJ), (interesting, JJ)..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]","[looks, like, you, just, started, it]","[looks, like, started]","[(looks, NNS), (like, IN), (started, VBD)]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]","[grr, lower]","[grr, lower]","[(grr, NNS), (lower, JJR)]"


In [14]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
messages_df['wordnet_pos'] = messages_df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,...","[why, no, milk, price, tracker, thread, we, ha...","[milk, price, tracker, thread, beef, lamb, fee...","[(milk, NN), (price, NN), (tracker, NN), (thre...","[(milk, n), (price, n), (tracker, n), (thread,..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno...","[christ, no, its, bad, enough, knowing, what, ...","[christ, bad, enough, knowing, getting, would,...","[(christ, NN), (bad, JJ), (enough, RB), (knowi...","[(christ, n), (bad, a), (enough, r), (knowing,..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, be, interesting, t...","[would, willing, interesting, see, variations,...","[(would, MD), (willing, JJ), (interesting, JJ)...","[(would, n), (willing, a), (interesting, a), (..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]","[looks, like, you, just, started, it]","[looks, like, started]","[(looks, NNS), (like, IN), (started, VBD)]","[(looks, n), (like, n), (started, v)]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]","[grr, lower]","[grr, lower]","[(grr, NNS), (lower, JJR)]","[(grr, n), (lower, a)]"


In [15]:
wnl = WordNetLemmatizer()
messages_df['lemmatized'] = messages_df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
messages_df.head()

,message,user_name,no_contract,no_contract_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,Why no milk price tracker thread? \nWe have th...,rc mx110,"[Why, no, milk, price, tracker, thread?, We, h...",Why no milk price tracker thread? We have them...,en,"[Why, no, milk, price, tracker, thread, ?, We,...","[why, no, milk, price, tracker, thread, ?, we,...","[why, no, milk, price, tracker, thread, we, ha...","[milk, price, tracker, thread, beef, lamb, fee...","[(milk, NN), (price, NN), (tracker, NN), (thre...","[(milk, n), (price, n), (tracker, n), (thread,...","[milk, price, tracker, thread, beef, lamb, fee..."
1,Christ no!!!! \nIts bad enough knowing what im...,buffalo_soldier,"[Christ, no!!!!, Its, bad, enough, knowing, wh...",Christ no!!!! Its bad enough knowing what i am...,en,"[Christ, no, !, !, !, !, Its, bad, enough, kno...","[christ, no, !, !, !, !, its, bad, enough, kno...","[christ, no, its, bad, enough, knowing, what, ...","[christ, bad, enough, knowing, getting, would,...","[(christ, NN), (bad, JJ), (enough, RB), (knowi...","[(christ, n), (bad, a), (enough, r), (knowing,...","[christ, bad, enough, know, get, would, even, ..."
2,"I would be willing to, be interesting to see v...",rc mx110,"[I, would, be, willing, to,, be, interesting, ...","I would be willing to, be interesting to see v...",en,"[I, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, ,, be, interesting...","[i, would, be, willing, to, be, interesting, t...","[would, willing, interesting, see, variations,...","[(would, MD), (willing, JJ), (interesting, JJ)...","[(would, n), (willing, a), (interesting, a), (...","[would, willing, interesting, see, variation, ..."
3,Looks like you just started it.,Robigus,"[Looks, like, you, just, started, it.]",Looks like you just started it.,en,"[Looks, like, you, just, started, it, .]","[looks, like, you, just, started, it, .]","[looks, like, you, just, started, it]","[looks, like, started]","[(looks, NNS), (like, IN), (started, VBD)]","[(looks, n), (like, n), (started, v)]","[look, like, start]"
4,\nGrr . Lower,buffalo_soldier,"[Grr, ., Lower]",Grr . Lower,en,"[Grr, ., Lower]","[grr, ., lower]","[grr, lower]","[grr, lower]","[(grr, NNS), (lower, JJR)]","[(grr, n), (lower, a)]","[grr, low]"


In [16]:
messages_df.to_pickle('messages_scraped_clean.pkl')